In [ ]:
from IPython.display import display, Markdown

In [ ]:
content = """
you're beautiful, you're beautiful, its true. i saw your face in a crowded place, and i dont know what to do. do you know this song?
"""


import ollama

response = ollama.chat(
    model='llama3.2:1b', # or your chosen model
    messages=[
        {
            'role': 'user',
            'content': content,
        },
    ],
)

display(Markdown(response['message']['content']))

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.2:1b",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field in Arlington, Texas, as the Los Angeles Dodgers faced the Tampa Bay Rays. The Dodgers came back from a 3-1 deficit to win the series 4 games to 2.


In [ ]:
from autogen import AssistantAgent, UserProxyAgent

config_list = [
  {
    "model": "codellama",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
  }
]

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False})
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

In [2]:
import psycopg2
from psycopg2.extras import execute_values
import requests
import json

# Ollama API endpoint
OLLAMA_API = "http://localhost:11434/api/embeddings"

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5444"
}

def get_embedding(text):
    response = requests.post(OLLAMA_API, json={"model": "all-minilm", "prompt": text})
    return response.json()['embedding']

def insert_documents(documents):
    conn = psycopg2.connect(**DB_PARAMS)
    cur = conn.cursor()

    try:
        data = [(doc['title'], doc['content'], get_embedding(doc['content'])) for doc in documents]
        execute_values(cur, """
            INSERT INTO documents (title, content, embedding)
            VALUES %s
        """, data)
        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()

# Example usage
documents = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}

]

insert_documents(documents)

In [3]:
import psycopg2
import requests
import json

# Ollama API endpoint
OLLAMA_API = "http://localhost:11434/api"

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5444"
}

def connect_db():
    return psycopg2.connect(**DB_PARAMS)

def get_embedding(text):
    response = requests.post(f"{OLLAMA_API}/embeddings", json={"model": "all-minilm", "prompt": text})
    return response.json()['embedding']

def generate_response(prompt):
    response = requests.post(f"{OLLAMA_API}/generate", json={
        "model": "llama3.2:1b",
        "prompt": prompt,
        "stream": False
    }, stream=True)
    
    full_response = ""
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if 'response' in json_response:
                full_response += json_response['response']
    
    return full_response

def retrieve_and_generate_response(query):
    conn = connect_db()
    cur = conn.cursor()
    
    # Embed the query
    query_embedding = get_embedding(query)
    
    # Format the embedding for pg_vector
    embedding_string = f"[{','.join(map(str, query_embedding))}]"

    print(embedding_string)
    
    # Retrieve relevant documents using cosine similarity
    cur.execute("""
        SELECT title, content, 1 - (embedding <=> %s::vector) AS similarity
        FROM documents
        ORDER BY similarity DESC
        LIMIT 5;
    """, (embedding_string,))
    
    rows = cur.fetchall()
    
    # Prepare the context for generating the response
    context = "\n\n".join([f"Title: {row[0]}\nContent: {row[1]}" for row in rows])
    
    # Generate the response
    prompt = f"Query: {query}\nContext: {context}\nPlease provide a concise answer based on the given context."
    response = generate_response(prompt)
    
    print(f"Response: {response}")
    
    cur.close()
    conn.close()

def main():
    retrieve_and_generate_response("Tell me about landmarks in Seoul")


main()

[0.3247070014476776,0.47870689630508423,0.4221107065677643,0.13946548104286194,0.09646524488925934,-0.5068508386611938,0.044916898012161255,0.09759426862001419,-0.6702886819839478,0.2098851501941681,0.00232620257884264,-0.2912425398826599,0.12144449353218079,-0.33253392577171326,0.26646822690963745,-0.15559035539627075,0.526840329170227,0.678272008895874,-0.01936432719230652,-0.7570167779922485,0.46246975660324097,-0.7943563461303711,0.2977140247821808,0.16200891137123108,-0.5540757775306702,-0.18961277604103088,0.24850451946258545,0.06526913493871689,0.3460521697998047,-0.2528931498527527,-0.04513245448470116,0.1523456573486328,0.20112156867980957,0.5428791642189026,-0.00011243671178817749,0.7022920846939087,0.10765612125396729,-0.021432343870401382,-0.05055682733654976,-0.1992662101984024,-0.37018030881881714,-0.12373881042003632,0.7185728549957275,-0.5715293884277344,0.6554132699966431,0.11795195937156677,-0.2620759606361389,-0.6693871021270752,-0.04215824604034424,-0.23633176088333